# AlexNet代码练习

### （1）**AlexNet_model**

In [4]:
import torch
import torch.nn as nn
from collections import OrderedDict  # 这个库方便给网络每一层添加名字。


class AlexNet(nn.Module):
    def __init__(self, num_classes=1000, init_weights=False):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(
                in_channels=3, out_channels=48, kernel_size=11, padding=2, stride=4
            ),  # input[3, 224, 224]  output[48, 55, 55]
            nn.ReLU(inplace=True),  # 参数表示是否会修改输入对象的值
            nn.MaxPool2d(kernel_size=3, stride=2),  # output[48, 27, 27]
            nn.Conv2d(
                in_channels=48, out_channels=128, kernel_size=5, padding=2
            ),  # output[128, 27, 27]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),  # output[128, 13, 13]
            nn.Conv2d(
                in_channels=128, out_channels=192, kernel_size=3, padding=1
            ),  # output[192, 13, 13]
            nn.ReLU(inplace=True),
            nn.Conv2d(
                in_channels=192, out_channels=192, kernel_size=3, padding=1
            ),  # output[192, 13, 13]
            nn.ReLU(inplace=True),
            nn.Conv2d(
                in_channels=192, out_channels=128, kernel_size=3, padding=1
            ),  # output[128, 13, 13]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),  # output[128, 6, 6]
        )

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),  # 神经元随机失活，避免过拟合的一种操作。
            nn.Linear(in_features=128 * 6 * 6, out_features=2048),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=2048, out_features=2048),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=2048, out_features=num_classes),
        )

        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        # 维度从0开始，将1到最后一个维度展平以输入全连接层。
        x = torch.flatten(x, start_dim=1, end_dim=-1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        """初始化权重"""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

In [6]:
net = AlexNet()
x = torch.rand([1, 3, 224, 224])
y = net(x)
print(y.shape)

torch.Size([1, 1000])


### （2）**train**

In [ ]:
# 训练前先下载数据，正确设置训练、验证的数据路径。

import os
import sys
import json

import torch
import torch.nn as nn
from torchvision import transforms, datasets
import torch.optim as optim
from tqdm import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用{}进行训练".format(device))

data_transform = {
    "train_transform": transforms.Compose(
        [
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    ),
    "val_transform": transforms.Compose(
        [
            transforms.Resize((224, 224)),  # cannot 224, must (224, 224)
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    ),
}

data_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))  # get data root path
image_path = os.path.join(data_root, "data_set", "flower_data")  # flower data set path
assert os.path.exists(image_path), "{} path does not exist.".format(image_path)
# datasets.ImageFolder()是加载自己的数据集，需要将数据分类放在不同的文件夹中，
# 文件夹名就是数据类别名。
train_dataset = datasets.ImageFolder(
    root=os.path.join(image_path, "train"), transform=data_transform["train_transform"]
)
train_num = len(train_dataset)

# 建立花类别的json文件
flower_list = train_dataset.class_to_idx
cla_dict = dict((val, key) for key, val in flower_list.items())
# write dict into json file
json_str = json.dumps(cla_dict, indent=4)
with open("class_indices.json", "w") as json_file:
    json_file.write(json_str)

# 确定用几个进程加载数据
batch_size = 32
nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
print("Using {} dataloader workers every process".format(nw))

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=nw
)

validate_dataset = datasets.ImageFolder(
    root=os.path.join(image_path, "val"), transform=data_transform["val_transform"]
)
val_num = len(validate_dataset)
validate_loader = torch.utils.data.DataLoader(
    validate_dataset, batch_size=4, shuffle=False, num_workers=nw
)

print(
    "using {} images for training, {} images for validation.".format(train_num, val_num)
)

alexnet = AlexNet(num_classes=5, init_weights=True)
net.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0002)

epochs = 10
save_path = "./AlexNet.pth"
best_acc = 0.0
train_steps = len(train_loader)

for epoch in range(epochs):
    # train
    net.train()
    running_loss = 0.0
    # 显示训练进度条
    train_bar = tqdm(train_loader, file=sys.stdout)
    for step, data in enumerate(train_bar):
        images, labels = data
        optimizer.zero_grad()
        outputs = net(images.to(device))
        loss = loss_function(outputs, labels.to(device))
        loss.backward()
        # 更新参数
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(
            epoch + 1, epochs, loss
        )

    # validate
    net.eval()
    acc = 0.0  # accumulate accurate number / epoch
    with torch.no_grad():
        val_bar = tqdm(validate_loader, file=sys.stdout)
        for val_data in val_bar:
            val_images, val_labels = val_data
            outputs = net(val_images.to(device))
            predict_y = torch.max(outputs, dim=1)[1]
            acc += torch.eq(predict_y, val_labels.to(device)).sum().item()

    val_accurate = acc / val_num
    print(
        "[epoch %d] train_loss: %.3f  val_accuracy: %.3f"
        % (epoch + 1, running_loss / train_steps, val_accurate)
    )

    if val_accurate > best_acc:
        best_acc = val_accurate
        torch.save(net.state_dict(), save_path)

print("Finished Training")

### （3）**predict**

In [ ]:
import os
import json

import torch
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt

from model import AlexNet

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

# load image
img_path = "../tulip.jpg"
assert os.path.exists(img_path), "file: '{}' dose not exist.".format(img_path)
img = Image.open(img_path)

plt.imshow(img)
# [N, C, H, W]
img = data_transform(img)
# expand batch dimension
img = torch.unsqueeze(img, dim=0)

# read class_indict
json_path = "./class_indices.json"
assert os.path.exists(json_path), "file: '{}' dose not exist.".format(json_path)

with open(json_path, "r") as f:
    class_indict = json.load(f)

# create model
model = AlexNet(num_classes=5).to(device)

# load model weights
weights_path = "./AlexNet.pth"
assert os.path.exists(weights_path), "file: '{}' dose not exist.".format(weights_path)
model.load_state_dict(torch.load(weights_path))

model.eval()
with torch.no_grad():
    # predict class
    output = torch.squeeze(model(img.to(device))).cpu()
    predict = torch.softmax(output, dim=0)
    predict_cla = torch.argmax(predict).numpy()

print_res = "class: {}   prob: {:.3}".format(
    class_indict[str(predict_cla)], predict[predict_cla].numpy()
)
plt.title(print_res)
for i in range(len(predict)):
    print("class: {:10}   prob: {:.3}".format(class_indict[str(i)], predict[i].numpy()))
plt.show()